## Back Testing

Below, we will use our algorithm to create a stock portfolio for 2022 and then see how it performed over the next year.

In [1]:
import numpy as np
import pandas as pd
import requests
# import xlsxwriter
import math
from scipy.stats import percentileofscore as score
from statistics import mean

In [2]:
sp500_stocks = pd.read_csv('constituents_backtesting.csv')

from mysecrets2 import POLYGON_API_KEY

In [3]:
backtest_columns = [
    'Ticker',
    'Price Today',
    'One-Year Price',
    'One-Year Return',
    'One-Year Percentile',
    'Six-Month Price',
    'Six-Month Return',
    'Six-Month Percentile',
    'Three-Month Price',
    'Three-Month Return',
    'Three-Month Percentile',
    'One-Month Price',
    'One-Month Return',
    'One-Month Percentile',
    'HQM Score',
    'Number of Shares to Buy'
]

backtest_df = pd.DataFrame(columns = backtest_columns)

In [4]:
cur_date = '2022-11-03'
oneyr_date = '2021-11-03'
sixmn_date = '2022-05-03'
threemn_date = '2022-08-03'
onemn_date = '2022-10-03'

for stk in sp500_stocks['Ticker'][:]:

    cur_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{cur_date}/{cur_date}?apiKey={POLYGON_API_KEY}'
    cur_data = requests.get(cur_api).json()
    cur_results0 = cur_data['results']
    cur_results1 = cur_results0[0]
    cur_price = cur_results1['c']

    oneyr_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{oneyr_date}/{oneyr_date}?apiKey={POLYGON_API_KEY}'
    oneyr_data = requests.get(oneyr_api).json()
    oneyr_results0 = oneyr_data['results']
    oneyr_results1 = oneyr_results0[0]
    oneyr_price = oneyr_results1['c']

    sixmn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{sixmn_date}/{sixmn_date}?apiKey={POLYGON_API_KEY}'
    sixmn_data = requests.get(sixmn_api).json()
    sixmn_results0 = sixmn_data['results']
    sixmn_results1 = sixmn_results0[0]
    sixmn_price = sixmn_results1['c']

    threemn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{threemn_date}/{threemn_date}?apiKey={POLYGON_API_KEY}'
    threemn_data = requests.get(threemn_api).json()
    threemn_results0 = threemn_data['results']
    threemn_results1 = threemn_results0[0]
    threemn_price = threemn_results1['c']

    onemn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{onemn_date}/{onemn_date}?apiKey={POLYGON_API_KEY}'
    onemn_data = requests.get(onemn_api).json()
    onemn_results0 = onemn_data['results']
    onemn_results1 = onemn_results0[0]
    onemn_price = onemn_results1['c']

    oneyr_return = (cur_price - oneyr_price) / oneyr_price
    sixmn_return = (cur_price - sixmn_price) / sixmn_price
    threemn_return = (cur_price - threemn_price) / threemn_price
    onemn_return = (cur_price - onemn_price) / onemn_price

    backtest_df = backtest_df.append(pd.Series([stk, 
                                      cur_price, 
                                      oneyr_price, 
                                      oneyr_return, 
                                      'N/A', 
                                      sixmn_price, 
                                      sixmn_return, 
                                      'N/A', 
                                      threemn_price, 
                                      threemn_return, 
                                      'N/A', 
                                      onemn_price, 
                                      onemn_return, 
                                      'N/A',
                                      'N/A',
                                      'N/A'], 
                                      index = backtest_columns), 
                                      ignore_index = True)

/tmp/ipykernel_540/2512854818.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  backtest_df = backtest_df.append(pd.Series([stk,
/tmp/ipykernel_540/2512854818.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  backtest_df = backtest_df.append(pd.Series([stk,
/tmp/ipykernel_540/2512854818.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  backtest_df = backtest_df.append(pd.Series([stk,
/tmp/ipykernel_540/2512854818.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  backtest_df = backtest_df.append(pd.Series([stk,
/tmp/ipykernel_540/2512854818.py:44: FutureWarning: The frame.append method is deprecated and will be removed from p

In [5]:
backtest_df

,Ticker,Price Today,One-Year Price,One-Year Return,One-Year Percentile,Six-Month Price,Six-Month Return,Six-Month Percentile,Three-Month Price,Three-Month Return,Three-Month Percentile,One-Month Price,One-Month Return,One-Month Percentile,HQM Score,Number of Shares to Buy
0,MMM,122.85,182.23,-0.325852,N/A,148.08,-0.170381,N/A,143.49,-0.143843,N/A,113.22,0.085056,N/A,N/A,N/A
1,AOS,54.37,78.39,-0.306417,N/A,59.10,-0.080034,N/A,61.48,-0.115647,N/A,50.63,0.073869,N/A,N/A,N/A
2,ABT,96.45,128.70,-0.250583,N/A,112.43,-0.142133,N/A,109.61,-0.120062,N/A,99.52,-0.030848,N/A,N/A,N/A
3,ABBV,144.42,117.57,0.228375,N/A,149.60,-0.034626,N/A,141.20,0.022805,N/A,138.32,0.044101,N/A,N/A,N/A
4,ACN,256.88,362.10,-0.290583,N/A,304.82,-0.157273,N/A,308.17,-0.166434,N/A,264.89,-0.030239,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,YUM,119.50,124.05,-0.036679,N/A,115.07,0.038498,N/A,119.81,-0.002587,N/A,109.11,0.095225,N/A,N/A,N/A
482,ZBRA,227.32,578.49,-0.607046,N/A,368.14,-0.382518,N/A,326.41,-0.303575,N/A,272.09,-0.164541,N/A,N/A,N/A
483,ZBH,105.42,148.74,-0.291246,N/A,120.68,-0.126450,N/A,115.61,-0.088141,N/A,107.32,-0.017704,N/A,N/A,N/A
484,ZION,48.12,65.78,-0.268471,N/A,57.47,-0.162694,N/A,53.96,-0.108228,N/A,52.21,-0.078337,N/A,N/A,N/A


In [6]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in backtest_df.index:
    for period in time_periods:
        backtest_df.loc[row, f'{period} Percentile'] = score(backtest_df[f'{period} Return'], backtest_df.loc[row, f'{period} Return'])

In [7]:
for row in backtest_df.index:
    momentum_perc = []
    
    for period in time_periods:
        momentum_perc.append(backtest_df.loc[row, f'{period} Percentile'])
    
    backtest_df.loc[row, 'HQM Score'] = mean(momentum_perc)

In [8]:
backtest_df.sort_values('HQM Score', ascending = False, inplace = True)
backtest_df = backtest_df[:50]
backtest_df.reset_index(inplace = True, drop = True)

In [9]:
portfolio_size = 1000000
position_size = float(portfolio_size) / len(backtest_df.index)

In [10]:
for i in range(0, len(backtest_df)):

    backtest_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / backtest_df.loc[i, 'Price Today'])

backtest_df

,Ticker,Price Today,One-Year Price,One-Year Return,One-Year Percentile,Six-Month Price,Six-Month Return,Six-Month Percentile,Three-Month Price,Three-Month Return,Three-Month Percentile,One-Month Price,One-Month Return,One-Month Percentile,HQM Score,Number of Shares to Buy
0,COP,133.82,72.74,0.839703,99.176955,98.93,0.352674,98.971193,91.32,0.465396,99.794239,109.97,0.216877,95.679012,98.40535,149
1,SLB,51.80,32.98,0.570649,97.119342,40.15,0.290162,96.91358,35.54,0.457513,99.588477,38.30,0.352480,99.382716,98.251029,386
2,HES,142.10,81.62,0.740995,98.559671,106.86,0.329777,98.353909,105.66,0.344880,98.559671,116.80,0.216610,95.473251,97.736626,140
3,MRO,31.42,16.49,0.905397,99.588477,26.25,0.196952,93.621399,22.89,0.372652,98.971193,24.97,0.258310,97.530864,97.427984,636
4,XOM,111.10,63.93,0.737838,98.353909,88.19,0.259780,95.473251,91.02,0.220611,95.679012,91.92,0.208660,94.650206,96.039095,180
5,FSLR,157.73,116.83,0.350081,93.621399,75.11,1.099987,100.0,97.89,0.611298,100.0,136.62,0.154516,89.300412,95.730453,126
6,GILD,79.46,67.41,0.178757,87.654321,60.33,0.317089,98.148148,62.27,0.276056,97.119342,62.32,0.275032,98.559671,95.37037,251
7,APA,47.29,27.63,0.711545,98.148148,42.52,0.112183,85.596708,34.43,0.373511,99.176955,37.30,0.267828,97.942387,95.216049,422
8,FANG,158.53,111.81,0.417852,94.855967,136.11,0.164720,91.563786,121.38,0.306064,97.736626,129.52,0.223981,96.502058,95.164609,126
9,MPC,118.20,63.68,0.856156,99.382716,92.65,0.275769,96.090535,92.68,0.275356,96.91358,102.79,0.149917,88.065844,95.113169,169


In [11]:
act_columns = ['Ticker', 'Price 11-03-2022', 'Price 11-03-2023', 'One-Year Return']

actual_df = pd.DataFrame(columns = act_columns)

In [12]:
start_date = '2022-11-03'
new_date = '2023-11-03'

for stk in backtest_df['Ticker']:

    print(stk)

    start_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{start_date}/{start_date}?apiKey={POLYGON_API_KEY}'
    start_data = requests.get(start_api).json()
    start_results0 = start_data['results']
    start_results1 = start_results0[0]
    start_price = start_results1['c']

    new_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{new_date}/{new_date}?apiKey={POLYGON_API_KEY}'
    new_data = requests.get(new_api).json()
    new_results0 = new_data['results']
    new_results1 = new_results0[0]
    new_price = new_results1['c']

    act_return = (new_price - start_price) / start_price

    actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)

COP


/tmp/ipykernel_540/700453086.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


SLB


/tmp/ipykernel_540/700453086.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


HES


...

/tmp/ipykernel_540/700453086.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


GL


/tmp/ipykernel_540/700453086.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


PXD


/tmp/ipykernel_540/700453086.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actual_df = actual_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


In [13]:
actual_df

,Ticker,Price 11-03-2022,Price 11-03-2023,One-Year Return
0,COP,133.82,119.75,-0.105141
1,SLB,51.80,56.87,0.097876
2,HES,142.10,146.26,0.029275
3,MRO,31.42,27.46,-0.126034
4,XOM,111.10,107.78,-0.029883
5,FSLR,157.73,151.94,-0.036708
6,GILD,79.46,81.59,0.026806
7,APA,47.29,39.96,-0.155001
8,FANG,158.53,162.10,0.022519
9,MPC,118.20,150.78,0.275635


In [14]:
average_return = actual_df['One-Year Return'].mean()

In [15]:
tot_columns = ['Ticker', 'Price 11-03-2022', 'Price 11-03-2023', 'One-Year Return']

total_df = pd.DataFrame(columns = tot_columns)

In [16]:
start_date = '2022-11-03'
new_date = '2023-11-03'

for stk in sp500_stocks['Ticker']:

    print(stk)

    start_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{start_date}/{start_date}?apiKey={POLYGON_API_KEY}'
    start_data = requests.get(start_api).json()
    start_results0 = start_data['results']
    start_results1 = start_results0[0]
    start_price = start_results1['c']

    new_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{new_date}/{new_date}?apiKey={POLYGON_API_KEY}'
    new_data = requests.get(new_api).json()
    new_results0 = new_data['results']
    new_results1 = new_results0[0]
    new_price = new_results1['c']

    act_return = (new_price - start_price) / start_price

    total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)

MMM


/tmp/ipykernel_540/17100239.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


AOS


/tmp/ipykernel_540/17100239.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


ABT


...

/tmp/ipykernel_540/17100239.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


ZION


/tmp/ipykernel_540/17100239.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


ZTS


/tmp/ipykernel_540/17100239.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(pd.Series([stk, start_price, new_price, act_return], index = act_columns), ignore_index = True)


We can see why it is so hard to consistently beat the market. The mean return for our portfolio is 1.42%, while the mean return for the S&P 500 as a whole was 6.67% during the year ended 11/03/2023.

In [17]:
print(average_return)
print(total_df['One-Year Return'].mean())

0.014193838350918298
0.06670367677669232
